# Integração Multissetorial de Dados do Piauí

## Demonstração do Projeto 4 - Portfólio Data Science

Este notebook demonstra a integração de dados públicos dos **224 municípios do Piauí**, combinando:

- **Dados Reais**: PIB e População (IBGE/SIDRA)
- **Dados Sintéticos**: Saúde, Educação, Assistência Social

### Tecnologias Utilizadas
- **API**: FastAPI com documentação Swagger
- **Dados**: IBGE SIDRA API + Faker (dados sintéticos)
- **Visualização**: Plotly, Matplotlib, Seaborn

---

In [ ]:
# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')

# Configurações de estilo
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Cores do tema
CORES = {
    'primaria': '#2E86AB',
    'secundaria': '#A23B72',
    'sucesso': '#18A558',
    'alerta': '#F18F01',
    'destaque': '#C73E1D'
}

print('✅ Bibliotecas carregadas com sucesso!')

## 1. Carregando os Dados Reais do IBGE

Os dados foram extraídos diretamente da **API SIDRA do IBGE**:
- **Tabela 6579**: Estimativas de População
- **Tabela 5938**: PIB dos Municípios

In [ ]:
# Carregar dados reais do IBGE
df_economia = pd.read_csv('../data/real/economia_completo.csv')
df_populacao = pd.read_csv('../data/real/populacao.csv')

print('=' * 60)
print('DADOS REAIS DO IBGE - PIAUÍ')
print('=' * 60)
print(f'\n📊 PIB Municipal: {len(df_economia):,} registros')
print(f'   • Municípios: {df_economia["municipio_id"].nunique()}')
print(f'   • Anos: {sorted(df_economia["ano"].unique())}')
print(f'\n👥 População: {len(df_populacao):,} registros')
print(f'   • Municípios: {df_populacao["municipio_id"].nunique()}')
print(f'   • Anos: {sorted(df_populacao["ano"].unique())}')

In [ ]:
# Visualizar estrutura dos dados
print('\n📝 Estrutura dos Dados de PIB:')
df_economia.head(10)

## 2. Visão Geral do Piauí

### 2.1 Indicadores Agregados (2021)

In [ ]:
# Calcular indicadores agregados para 2021
df_2021 = df_economia[df_economia['ano'] == 2021].copy()

# Calcular PIB per capita
df_2021['pib_per_capita'] = (df_2021['pib_total_mil_reais'] * 1000) / df_2021['populacao_estimada']

# Métricas
pib_total_pi = df_2021['pib_total_mil_reais'].sum() / 1_000_000  # Em bilhões
pop_total_pi = df_2021['populacao_estimada'].sum()
pib_per_capita_medio = df_2021['pib_per_capita'].mean()
pib_per_capita_mediano = df_2021['pib_per_capita'].median()

# Card de métricas
fig = make_subplots(
    rows=1, cols=4,
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, 
            {'type': 'indicator'}, {'type': 'indicator'}]]
)

fig.add_trace(go.Indicator(
    mode="number",
    value=pib_total_pi,
    title={'text': "PIB Total (Bilhões R$)", 'font': {'size': 14}},
    number={'prefix': "R$ ", 'suffix': " bi", 'font': {'size': 36, 'color': CORES['primaria']}}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=pop_total_pi,
    title={'text': "População Total", 'font': {'size': 14}},
    number={'font': {'size': 36, 'color': CORES['secundaria']}}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="number",
    value=pib_per_capita_medio,
    title={'text': "PIB per Capita Médio", 'font': {'size': 14}},
    number={'prefix': "R$ ", 'font': {'size': 36, 'color': CORES['sucesso']}}
), row=1, col=3)

fig.add_trace(go.Indicator(
    mode="number",
    value=224,
    title={'text': "Municípios", 'font': {'size': 14}},
    number={'font': {'size': 36, 'color': CORES['alerta']}}
), row=1, col=4)

fig.update_layout(
    title={'text': '🎯 Indicadores do Piauí - 2021 (Dados Reais IBGE)', 'x': 0.5, 'font': {'size': 20}},
    height=200,
    margin=dict(t=80, b=20, l=20, r=20)
)

fig.show()

## 3. Análise do PIB Municipal

### 3.1 Top 10 Maiores Economias do Piauí

In [ ]:
# Top 10 PIB 2021
top_10_pib = df_2021.nlargest(10, 'pib_total_mil_reais').copy()
top_10_pib['pib_bilhoes'] = top_10_pib['pib_total_mil_reais'] / 1_000_000

fig = px.bar(
    top_10_pib,
    x='pib_bilhoes',
    y='municipio_nome',
    orientation='h',
    color='pib_bilhoes',
    color_continuous_scale='Blues',
    text=top_10_pib['pib_bilhoes'].apply(lambda x: f'R$ {x:.1f} bi')
)

fig.update_layout(
    title={'text': '🏆 Top 10 Maiores Economias do Piauí (2021)', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='PIB (Bilhões R$)',
    yaxis_title='',
    yaxis={'categoryorder': 'total ascending'},
    height=500,
    showlegend=False,
    coloraxis_showscale=False
)

fig.update_traces(textposition='outside')
fig.show()

# Percentual de concentração
pib_top10 = top_10_pib['pib_total_mil_reais'].sum()
pib_total = df_2021['pib_total_mil_reais'].sum()
print(f'\n📊 Concentração Econômica:')
print(f'   Os 10 maiores municípios concentram {pib_top10/pib_total*100:.1f}% do PIB estadual')

### 3.2 Distribuição do PIB per Capita

In [ ]:
# Distribuição do PIB per capita
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Histograma do PIB per Capita', 'Box Plot por Faixa de População'),
    column_widths=[0.5, 0.5]
)

# Histograma
fig.add_trace(
    go.Histogram(
        x=df_2021['pib_per_capita'],
        nbinsx=30,
        marker_color=CORES['primaria'],
        opacity=0.7,
        name='Distribuição'
    ),
    row=1, col=1
)

# Adicionar linha da média
fig.add_vline(
    x=pib_per_capita_medio, 
    line_dash="dash", 
    line_color=CORES['destaque'],
    annotation_text=f"Média: R$ {pib_per_capita_medio:,.0f}",
    row=1, col=1
)

# Box plot por faixa de população
df_2021['faixa_pop'] = pd.cut(
    df_2021['populacao_estimada'],
    bins=[0, 5000, 10000, 20000, 50000, 100000, 1000000],
    labels=['<5k', '5k-10k', '10k-20k', '20k-50k', '50k-100k', '>100k']
)

for faixa in df_2021['faixa_pop'].cat.categories:
    dados_faixa = df_2021[df_2021['faixa_pop'] == faixa]['pib_per_capita']
    fig.add_trace(
        go.Box(y=dados_faixa, name=faixa, marker_color=CORES['secundaria']),
        row=1, col=2
    )

fig.update_layout(
    title={'text': '💰 Distribuição do PIB per Capita nos Municípios (2021)', 'x': 0.5, 'font': {'size': 18}},
    height=450,
    showlegend=False
)

fig.update_xaxes(title_text='PIB per Capita (R$)', row=1, col=1)
fig.update_yaxes(title_text='Frequência', row=1, col=1)
fig.update_xaxes(title_text='Faixa de População', row=1, col=2)
fig.update_yaxes(title_text='PIB per Capita (R$)', row=1, col=2)

fig.show()

### 3.3 Evolução do PIB (2020 vs 2021)

In [ ]:
# Calcular crescimento do PIB
df_2020 = df_economia[df_economia['ano'] == 2020][['municipio_id', 'municipio_nome', 'pib_total_mil_reais']].copy()
df_2020.columns = ['municipio_id', 'municipio_nome', 'pib_2020']

df_crescimento = df_2021[['municipio_id', 'municipio_nome', 'pib_total_mil_reais', 'populacao_estimada']].copy()
df_crescimento.columns = ['municipio_id', 'municipio_nome', 'pib_2021', 'populacao']

df_crescimento = df_crescimento.merge(df_2020[['municipio_id', 'pib_2020']], on='municipio_id')
df_crescimento['crescimento_pct'] = ((df_crescimento['pib_2021'] - df_crescimento['pib_2020']) / df_crescimento['pib_2020']) * 100

# Top 10 maiores crescimentos
top_crescimento = df_crescimento.nlargest(15, 'crescimento_pct')

fig = px.bar(
    top_crescimento,
    x='municipio_nome',
    y='crescimento_pct',
    color='crescimento_pct',
    color_continuous_scale='RdYlGn',
    text=top_crescimento['crescimento_pct'].apply(lambda x: f'+{x:.1f}%')
)

fig.update_layout(
    title={'text': '📈 Top 15 Municípios com Maior Crescimento do PIB (2020-2021)', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='',
    yaxis_title='Crescimento (%)',
    xaxis_tickangle=-45,
    height=500,
    coloraxis_showscale=False
)

fig.update_traces(textposition='outside')
fig.show()

# Estatísticas de crescimento
print(f'\n📊 Estatísticas de Crescimento do PIB (2020-2021):')
print(f'   • Crescimento médio: {df_crescimento["crescimento_pct"].mean():.1f}%')
print(f'   • Crescimento mediano: {df_crescimento["crescimento_pct"].median():.1f}%')
print(f'   • Maior crescimento: {df_crescimento["crescimento_pct"].max():.1f}% ({df_crescimento.loc[df_crescimento["crescimento_pct"].idxmax(), "municipio_nome"]})')
print(f'   • Menor crescimento: {df_crescimento["crescimento_pct"].min():.1f}% ({df_crescimento.loc[df_crescimento["crescimento_pct"].idxmin(), "municipio_nome"]})')

## 4. Análise Populacional

### 4.1 Evolução da População por Ano

In [ ]:
# População total por ano
pop_por_ano = df_populacao.groupby('ano')['populacao_estimada'].sum().reset_index()

fig = px.line(
    pop_por_ano,
    x='ano',
    y='populacao_estimada',
    markers=True,
    text=pop_por_ano['populacao_estimada'].apply(lambda x: f'{x/1_000_000:.2f}M')
)

fig.update_traces(
    line_color=CORES['primaria'],
    marker_size=12,
    textposition='top center',
    line_width=3
)

fig.update_layout(
    title={'text': '👥 Evolução da População do Piauí', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='Ano',
    yaxis_title='População',
    height=400,
    xaxis=dict(tickmode='array', tickvals=pop_por_ano['ano'].tolist())
)

fig.show()

### 4.2 Distribuição por Porte do Município

In [ ]:
# Classificar municípios por porte
df_2021_pop = df_populacao[df_populacao['ano'] == 2021].copy()

def classificar_porte(pop):
    if pop < 5000:
        return 'Muito Pequeno (<5k)'
    elif pop < 10000:
        return 'Pequeno (5k-10k)'
    elif pop < 20000:
        return 'Médio-Pequeno (10k-20k)'
    elif pop < 50000:
        return 'Médio (20k-50k)'
    elif pop < 100000:
        return 'Médio-Grande (50k-100k)'
    else:
        return 'Grande (>100k)'

df_2021_pop['porte'] = df_2021_pop['populacao_estimada'].apply(classificar_porte)

# Contagem por porte
porte_count = df_2021_pop['porte'].value_counts().reset_index()
porte_count.columns = ['porte', 'quantidade']

# Ordenar
ordem_porte = ['Muito Pequeno (<5k)', 'Pequeno (5k-10k)', 'Médio-Pequeno (10k-20k)', 
               'Médio (20k-50k)', 'Médio-Grande (50k-100k)', 'Grande (>100k)']
porte_count['porte'] = pd.Categorical(porte_count['porte'], categories=ordem_porte, ordered=True)
porte_count = porte_count.sort_values('porte')

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'pie'}, {'type': 'bar'}]],
    subplot_titles=('Distribuição por Porte', 'Quantidade de Municípios')
)

# Pizza
fig.add_trace(
    go.Pie(
        labels=porte_count['porte'],
        values=porte_count['quantidade'],
        hole=0.4,
        marker_colors=px.colors.sequential.Blues_r
    ),
    row=1, col=1
)

# Barras
fig.add_trace(
    go.Bar(
        x=porte_count['porte'],
        y=porte_count['quantidade'],
        marker_color=CORES['secundaria'],
        text=porte_count['quantidade'],
        textposition='outside'
    ),
    row=1, col=2
)

fig.update_layout(
    title={'text': '🏘️ Distribuição dos Municípios por Porte (2021)', 'x': 0.5, 'font': {'size': 18}},
    height=450,
    showlegend=False
)

fig.update_xaxes(tickangle=-45, row=1, col=2)

fig.show()

print(f'\n📊 Resumo por Porte:')
for _, row in porte_count.iterrows():
    print(f'   • {row["porte"]}: {row["quantidade"]} municípios ({row["quantidade"]/224*100:.1f}%)')

## 5. Relação PIB x População

### 5.1 Scatter Plot Interativo

In [ ]:
# Scatter plot PIB x População
df_scatter = df_2021.copy()
df_scatter['pib_bilhoes'] = df_scatter['pib_total_mil_reais'] / 1_000_000
df_scatter['pop_milhares'] = df_scatter['populacao_estimada'] / 1_000

fig = px.scatter(
    df_scatter,
    x='populacao_estimada',
    y='pib_total_mil_reais',
    size='pib_per_capita',
    color='pib_per_capita',
    hover_name='municipio_nome',
    hover_data={
        'populacao_estimada': ':,.0f',
        'pib_total_mil_reais': ':,.0f',
        'pib_per_capita': ':,.2f'
    },
    color_continuous_scale='Viridis',
    size_max=50,
    log_x=True,
    log_y=True
)

# Destacar Teresina
teresina = df_scatter[df_scatter['municipio_nome'] == 'Teresina'].iloc[0]
fig.add_annotation(
    x=np.log10(teresina['populacao_estimada']),
    y=np.log10(teresina['pib_total_mil_reais']),
    text="Teresina",
    showarrow=True,
    arrowhead=2,
    ax=40,
    ay=-40
)

fig.update_layout(
    title={'text': '📊 Relação PIB x População (2021) - Escala Logarítmica', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='População (escala log)',
    yaxis_title='PIB em Mil R$ (escala log)',
    height=600,
    coloraxis_colorbar_title='PIB per Capita'
)

fig.show()

### 5.2 Correlação entre Variáveis

In [ ]:
# Matriz de correlação
cols_corr = ['pib_total_mil_reais', 'populacao_estimada', 'pib_per_capita']
corr_matrix = df_2021[cols_corr].corr()

# Renomear para exibição
labels = ['PIB Total', 'População', 'PIB per Capita']

fig = px.imshow(
    corr_matrix,
    x=labels,
    y=labels,
    color_continuous_scale='RdBu_r',
    zmin=-1, zmax=1,
    text_auto='.2f'
)

fig.update_layout(
    title={'text': '🔗 Matriz de Correlação', 'x': 0.5, 'font': {'size': 18}},
    height=400,
    width=500
)

fig.show()

## 6. Destaques: Teresina (Capital)

### Comparação com o Interior

In [ ]:
# Dados de Teresina
teresina_data = df_economia[df_economia['municipio_nome'] == 'Teresina']
interior_data = df_economia[df_economia['municipio_nome'] != 'Teresina']

# Comparativo
teresina_2021 = teresina_data[teresina_data['ano'] == 2021].iloc[0]
interior_2021 = interior_data[interior_data['ano'] == 2021]

# Métricas comparativas
metricas = {
    'Métrica': ['PIB Total (R$ bi)', 'População', 'PIB per Capita (R$)', '% do PIB Estadual'],
    'Teresina': [
        f"{teresina_2021['pib_total_mil_reais']/1_000_000:.1f}",
        f"{teresina_2021['populacao_estimada']:,.0f}",
        f"{(teresina_2021['pib_total_mil_reais']*1000)/teresina_2021['populacao_estimada']:,.0f}",
        f"{teresina_2021['pib_total_mil_reais']/df_2021['pib_total_mil_reais'].sum()*100:.1f}%"
    ],
    'Interior (Média)': [
        f"{interior_2021['pib_total_mil_reais'].mean()/1_000_000:.3f}",
        f"{interior_2021['populacao_estimada'].mean():,.0f}",
        f"{((interior_2021['pib_total_mil_reais']*1000)/interior_2021['populacao_estimada']).mean():,.0f}",
        f"{interior_2021['pib_total_mil_reais'].sum()/df_2021['pib_total_mil_reais'].sum()*100:.1f}%"
    ]
}

df_comparativo = pd.DataFrame(metricas)

# Tabela formatada
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>Métrica</b>', '<b>Teresina</b>', '<b>Interior (Média)</b>'],
        fill_color=CORES['primaria'],
        font=dict(color='white', size=14),
        align='center',
        height=40
    ),
    cells=dict(
        values=[df_comparativo[col] for col in df_comparativo.columns],
        fill_color=[['#f0f0f0', 'white']*2],
        font=dict(size=13),
        align='center',
        height=35
    )
)])

fig.update_layout(
    title={'text': '🏙️ Teresina vs Interior do Piauí (2021)', 'x': 0.5, 'font': {'size': 18}},
    height=300
)

fig.show()

print(f'\n📊 Insight: Teresina concentra {teresina_2021["pib_total_mil_reais"]/df_2021["pib_total_mil_reais"].sum()*100:.1f}% do PIB estadual')
print(f'          com apenas {teresina_2021["populacao_estimada"]/df_2021["populacao_estimada"].sum()*100:.1f}% da população')

## 7. Mapa de Calor: PIB per Capita por Município

In [ ]:
# Treemap do PIB
df_treemap = df_2021.copy()
df_treemap['pib_bilhoes'] = df_treemap['pib_total_mil_reais'] / 1_000_000

# Adicionar classificação
df_treemap['faixa_pib_pc'] = pd.cut(
    df_treemap['pib_per_capita'],
    bins=[0, 10000, 15000, 20000, 30000, 100000],
    labels=['<R$10k', 'R$10k-15k', 'R$15k-20k', 'R$20k-30k', '>R$30k']
)

fig = px.treemap(
    df_treemap.nlargest(50, 'pib_total_mil_reais'),
    path=['faixa_pib_pc', 'municipio_nome'],
    values='pib_total_mil_reais',
    color='pib_per_capita',
    color_continuous_scale='RdYlGn',
    hover_data={'pib_per_capita': ':,.0f', 'populacao_estimada': ':,.0f'}
)

fig.update_layout(
    title={'text': '🗺️ Top 50 Municípios - PIB por Faixa de Renda per Capita', 'x': 0.5, 'font': {'size': 18}},
    height=600,
    coloraxis_colorbar_title='PIB per Capita (R$)'
)

fig.show()

## 8. Conclusões

### Principais Insights dos Dados Reais do IBGE:

In [ ]:
# Gerar insights automáticos
print('=' * 70)
print('PRINCIPAIS INSIGHTS - DADOS REAIS DO IBGE (2021)')
print('=' * 70)

print(f'''
💰 ECONOMIA:
   • PIB Total do Piauí: R$ {pib_total_pi:.1f} bilhões
   • Crescimento médio 2020-2021: {df_crescimento["crescimento_pct"].mean():.1f}%
   • PIB per capita médio: R$ {pib_per_capita_medio:,.0f}
   • PIB per capita mediano: R$ {pib_per_capita_mediano:,.0f}

🏙️ CONCENTRAÇÃO:
   • Teresina concentra {teresina_2021['pib_total_mil_reais']/df_2021['pib_total_mil_reais'].sum()*100:.1f}% do PIB estadual
   • Top 10 municípios: {pib_top10/pib_total*100:.1f}% do PIB
   • Desigualdade: PIB per capita varia de R$ {df_2021['pib_per_capita'].min():,.0f} a R$ {df_2021['pib_per_capita'].max():,.0f}

👥 POPULAÇÃO:
   • Total: {pop_total_pi:,} habitantes
   • {len(df_2021_pop[df_2021_pop['populacao_estimada'] < 10000])} municípios com menos de 10 mil habitantes ({len(df_2021_pop[df_2021_pop['populacao_estimada'] < 10000])/224*100:.0f}%)
   • Apenas {len(df_2021_pop[df_2021_pop['populacao_estimada'] > 100000])} município(s) com mais de 100 mil habitantes

📈 DESTAQUES:
   • Maior economia: {df_2021.loc[df_2021['pib_total_mil_reais'].idxmax(), 'municipio_nome']} (R$ {df_2021['pib_total_mil_reais'].max()/1_000_000:.1f} bi)
   • Maior PIB per capita: {df_2021.loc[df_2021['pib_per_capita'].idxmax(), 'municipio_nome']} (R$ {df_2021['pib_per_capita'].max():,.0f})
   • Maior crescimento: {df_crescimento.loc[df_crescimento['crescimento_pct'].idxmax(), 'municipio_nome']} (+{df_crescimento['crescimento_pct'].max():.1f}%)
''')

print('=' * 70)
print('Fonte: IBGE - Sistema de Contas Regionais (SIDRA)')
print('=' * 70)

---

## 🚀 Próximos Passos

Este projeto demonstra a integração de dados reais do IBGE com uma API REST. 

**Para testar a API:**
```bash
cd projeto4-integracao-multissetorial
python main.py api
```

Acesse: http://localhost:8000/docs

**Endpoints disponíveis:**
- `GET /economia/pib` - Dados reais do IBGE
- `GET /indicadores/{municipio_id}` - Indicadores consolidados
- `GET /fontes/status` - Status das fontes de dados

---

*Desenvolvido como parte do Portfólio de Data Science*